<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Reinforcement Learning <br>
<font color=696880 size=4>
    Mohammad Moshtaghi - Ali Salesi - Hossein Goli

____

# Personal Data

In [1]:
student_number = '400104867'
first_name = 'Payam'
last_name = 'Taebi'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q2: Sentence Generator (100 Points)

<font size=4>
Author: Ali Salesi
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

In this assignment we implement a text generator using RL.

## Preprocess

### Dataset

First, lets download the text corpus crawled from `VOA Persian` from 2003 to 2008.

In [2]:
!wget -O "voa_persian.txt" "https://storage.googleapis.com/danielk-files/farsi-text/merged_files/voa_persian_2003_2008_cleaned.txt"

--2023-05-09 18:35:05--  https://storage.googleapis.com/danielk-files/farsi-text/merged_files/voa_persian_2003_2008_cleaned.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 142.251.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69708061 (66M) [text/plain]
Saving to: ‘voa_persian.txt’

voa_persian.txt     100%[===================>]  66.48M  53.1MB/s    in 1.3s    

2023-05-09 18:35:06 (53.1 MB/s) - ‘voa_persian.txt’ saved [69708061/69708061]



In [3]:
!wc -l voa_persian.txt | awk '{print $1}'

488253


In [4]:
!head voa_persian.txt

پيمان صلح بين ژاپن و روسيه
بنا به گزارشهای منتشره در توکيو، ژاپن و روسيه در زمينه يک پيمان صلح در چارچوبی گسترده توافق کرده اند که رسماً به مخاصمات جنگ دوم جهانی ميان دو کشور پايان خواهند داد.

در يکی از اين گزارشها، که از سوی خبرگزاری کيودُو،انتشار يافته، گفته شده است که دو کشور برای رفع اختلافات ديرين خود بر سر چهار جزيره از جزاير زنجيره ای کوريل، بر اساس سه پيمان گذشته خود عمل خواهند کرد.
بموجب يکی از اين پيمانها که در سال ۱۹۵۶ امضاء شده، دو تا از اين جزيره ها پس از امضاء يک پيمان صلح به ژاپن پس داده خواهد شد.
اما بموجب پيمانی که در سال ۱۹۹۳ به امضاء رسيده، مسئله حاکميت اين چهار جزيره بايستی پيش از امضاء پيمان صلح فيصله يابد.
هيچ يک از دو طرف نحوه استفاده از پيمان های پيشين را اعلام نکرده اند.

تشکيلات فلسطينی نخستين بودجه رسمی خود را اعلام کرد
تشکيلات فلسطينی پس از دو سال نخستين بودجه رسمی خود را اعلام کرد و قول داد برای از ميان برداشتن فساد و پاسخگوئی بيشتر به مردم تلاش کند.


### Normalization

Then we have to normalize and lemmatize the text so we can have a better generalization of semantics in prompt generation.

We'll use `hazm` library for this purpose.

In [5]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394471 sha256=f01da08e3100e614fd33f2cf479b7394517e98e3f832bd86ba4c6232cfdf36f6
  Stored in directory: /root/.cache/pip/wheels/6b/6d/14/3defa4cd7013faeddf715150696f4a96d7725c87700eb8a68e
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp310-cp310-linux_x86_64.whl size=180366 sha256=1b7b6709e5cb5cedceea612b53703c44a6714ab448165c0b43dca14f71c9e5e6
  Stored in directory: /root/.cache/pip/wheels/9f/cb/30/fef48ecac051e433987eccdb5682900b4c00d44a4bcd4d4ec8
Successfully built nltk libwapiti
  Attempting un

In [6]:
from __future__ import unicode_literals
from hazm import Normalizer, Lemmatizer, word_tokenize
from tqdm import tqdm
import re

normalizer = Normalizer()
lemmatizer = Lemmatizer()


def normalize(line: str):
    line = re.sub(
        r'[.{}[\]؛:«»؟!٬٫٪×،*)(ـ+<>\'",`=+\-?!@#$%^&*()_\/\\\\]', '', line.strip())
    line = re.sub(r'\s+', ' ', line.strip())
    line = normalizer.normalize(line)
    words = word_tokenize(line)
    words = [lemmatizer.lemmatize(word) for word in words]
    line = ' '.join(words)
    return line


In [7]:
normalize('من خیلی خوشحال هستم و کتاب‌های زیادی درباره یخچال‌های قطبی خوانده‌ام.')

'من خیلی خوشحال #هست و کتاب زیاد درباره یخچال قطب خواند#خوان'

In [8]:
voa = open('voa_persian.txt')
voa_norm = open('voa_persian_normalized.txt', 'w')
for i, line in tqdm(enumerate(voa), total=488253):
    voa_norm.write(normalize(line) + '\n')


100%|██████████| 488253/488253 [01:06<00:00, 7326.02it/s]


In [9]:
!head voa_persian_normalized.txt

پیمان صلح بین ژاپن و روسیه
بنا به گزارش منتشره در توکیو ژاپن و روسیه در زمینه یک پیمان صلح در چارچوب گسترده توافق کرد#کن که رسما به مخاصمات جنگ دوم جهانی میان دو کشور پایان داد#ده

در یک از این گزارش که از سو خبرگزاری کیودوانتشار یافته گفت#گو که دو کشور برای رفع اختلافات دیرین خود بر سر چهار جزیره از جزایر زنجیره کوریل بر اساس سه پیمان گذشته خود عمل کرد#کن
بموجب یک از این پیمان که در سال ۱۹۵۶ امضاء شده دو تا از این جزیره پس از امضاء یک پیمان صلح به ژاپن پس داد#ده
اما بموجب پیمان که در سال ۱۹۹۳ به امضاء رسیده مسئله حاکمیت این چهار جزیره ایستاد#ایست پیش از امضاء پیمان صلح فیصله یافت#یاب
هیچ یک از دو طرف نحوه استفاده از پیمان پیشین را اعلام کرد#کن

تشکیلات فلسطین نخستین بودجه رسم خود را اعلام کرد#کن
تشکیلات فلسطین پس از دو سال نخستین بودجه رسم خود را اعلام کرد#کن و قول داد برای از میان برداشتن فساد و پاسخگوئی بیشتر به مردم تلاش کند


### Language Model

Now we'll use `KenLM` to train an N-gram language model. an N-gram model calculates probability of N words being together.

You can read more about N-gram [here](https://towardsdatascience.com/understanding-word-n-grams-and-n-gram-probability-in-natural-language-processing-9d9eef0fa058).

First, let's install download and build `KenLM`.

In [10]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz; mkdir kenlm/build; cd kenlm/build; cmake ..; make -j2

--2023-05-09 18:36:37--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  --.-KB/s    in 0.03s   

2023-05-09 18:36:37 (15.1 MB/s) - written to stdout [491888/491888]

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eige

Now let's make a 5-gram model using 

In [11]:
!kenlm/build/bin/lmplz -o 5 <"voa_persian_normalized.txt"> "voa_persian.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/voa_persian_normalized.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 7151282 types 105479
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1265748 2:1062613376 3:1992400128 4:3187840000 5:4648933888
Statistics:
1 105479 D1=0.692798 D2=1.02059 D3+=1.36868
2 1273831 D1=0.753634 D2=1.09875 D3+=1.3404
3 3442840 D1=0.837136 D2=1.17748 D3+=1.39394
4 5019073 D1=0.905517 D2=1.28916 D3+=1.43789
5 5610872 D1=0.891831 D2=1.51472 D3+=1.61131
Memory estimate for binary LM:
type     MB
probing 321 assuming -p 1.5
probing 377 assuming -r models -p 1.5
trie    153 without quantization
trie     83 assuming -q 8 -b 8 quantization 
trie    135 assuming -a 22 array pointer compression
trie     66 assuming -a 22 -q 8 -b 8 array pointer compression and quantiza

In [12]:
!head -n 20 voa_persian.arpa

\data\
ngram 1=105479
ngram 2=1273831
ngram 3=3442840
ngram 4=5019073
ngram 5=5610872

\1-grams:
-6.138535	<unk>	0
0	<s>	-1.5815679
-2.1129756	</s>	0
-3.5871809	پیمان	-0.50824106
-3.304699	صلح	-0.560521
-2.891446	بین	-0.67797303
-3.303326	ژاپن	-0.5074899
-2.0037236	و	-0.8103652
-3.097553	روسیه	-0.56382215
-3.694238	بنا	-0.5076225
-2.0797832	به	-1.0190648
-3.047614	گزارش	-0.6365477


Now lets extract the list of words and sort them using their probabilities.

In [13]:
words = []
words_started = False
with open('voa_persian.arpa') as f:
    for line in f:
        line = line.strip()
        if not words_started:
            if line == r'\1-grams:':
                words_started = True
        else:
            if line == r'\2-grams:':
                words = words[:-1]
                break
            words.append(line.split())
words_sorted = sorted(words, key=lambda x: x[0])
words_total = [w[1] for w in words_sorted]
words_total.remove('</s>')
words_total.insert(0, '</s>')
words_total[:20]


['</s>',
 'در',
 'و',
 'به',
 'را',
 'که',
 'از',
 'با',
 '#است',
 'بود#باش',
 'یک',
 'برای',
 'این',
 'شد#شو',
 'گفت#گو',
 'خود',
 'آن',
 'کرد#کن',
 'روز',
 'نیز']

In [14]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp310-cp310-linux_x86_64.whl size=3255085 sha256=c0ac5af41999b92be9f2a55dc2862483dbda538b1be88fa9a3b2d17f6f04b57f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ftqlch6n/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [15]:
import kenlm

model = kenlm.Model('voa_persian.arpa')

Now we need a measure using our language model to measure how well our sentence fit together. Our model can measure the probability of a sentence using N-gram.

This has a downside. the longer the sentence gets, the lower its' probability becomes. We don't want that. So we introduce `perplexity`. a measure which is normalized by the sentence's length. Lower perplexity means the semantics of our sentence fits better together.

You can read more about perplexity [here](https://medium.com/nlplanet/two-minutes-nlp-perplexity-explained-with-simple-probabilities-6cdc46884584).
$$
\begin{align}
PP(S) &= 10 ^ {-\frac{log(P(S))}{N}} \\
PP(S) &= \sqrt[N]{\frac{1}{P(S)}} \\
PP(S) &= \sqrt[N]{\frac{1}{P(W_1W_2...W_N)}} \\
PP(S) &= \sqrt[N]{\prod_{i=1}^N{\frac{1}{P(W_i|W_1W_2...W_{i-1})}}}
\end{align}
$$
**Note**: `KenLM` score function return log10 probability of a sentence.

### Perplexity (10 Points)

In [16]:
def perplexity(sentence: str):
    """
    returns the perplexity of a sentence using model.score method
    Args:
      sentence: string of words
      
    Returns:
      perplexity: 10^(-lop10p(sentence) / N)
    """
    return 10 ** (-model.score(sentence) / (len(sentence.split()) + 1) )

In [17]:
sen_1 = normalize('من خوشحال شدم')
sen_2 = normalize('من خودکار شدم')
sen_3 = normalize('من کتاب یخچال')
sen_4 = normalize('نستب سنبتس سنمبتم')


In [18]:
print(sen_1, perplexity(sen_1))
print(sen_2, perplexity(sen_2))
print(sen_3, perplexity(sen_3))
print(sen_4, perplexity(sen_4))


من خوشحال شد#شو 137.01635002572476
من خودکار شد#شو 1222.8152856084316
من کتاب یخچال 7466.752430895604
نستب سنبتس سنمبتم 336928.1876517719


## Reinforcement Learning

### Reward Function (10 Points)

Reward function should give us a reward based on how the last word added to the sentence changed the meaning and how well it fits with the others.

In [19]:
def reward(base_sentence: str, new_word: str):
    """
    returns the reward of adding a new word to a base sentence
    Args:
      base_sentence: string of words up until now
      new_word: new word to be added to the base sentence

    Returns:
      reward: change of perplexity of the base sentence after adding the new word. positive reward means the new word is more likely to be added to the base sentence.
    """
    return  - perplexity(base_sentence+ " " + new_word) + perplexity(base_sentence)


In [20]:
print(reward('', 'من'))
print(reward('من', 'خوشحال'))
print(reward('من خوشحال', 'شد#شو'))
print(reward('جنگ جهانی', 'اول'))
print(reward('جنگ جهانی', 'دوم'))
print(reward('جنگ جهانی', 'صورتی'))


-690.0276008581808
-167.83652051799663
724.0299139733988
457.49453326104816
482.9573756304417
-4980.644139684355


Since we have to implement text generator using a tabular implementation, we have to assume that all that matters in a text is in a window of N words. It matches our language model of N-gram.

We model it using MDP. the first state is `<s>` state. it has no text and 0 perplexity. The next state is $W_1$ state. We usually have a negative perplexity because no text has more meaning than a one word sentence. Next is $W_1W_2$ state until we reach $W_1W_2...W_N$ state, from then with our window assumption we go to $W_2W_3...W_{N+1}$ state and $W_3W_4...W_{N+2}$ and so on.

First thing we notice is that our search space is **really** big. Each word choice has thousands of possibilites. We cannot model our search space using our normal Q Table.
Since our states are sequential and we need to find the best word using our current state, we can use `dict` in `dict` architecture.

First we reduce the search space to the 10K most used words.
For faster computation, we use each word index for states.

### Utility Functions (10 Points)

In [21]:
words = words_total[:10000]
# 0 index is for </s> which means end of the sentence.
indexes = dict()
for i, w in enumerate(words):
    indexes[w] = i


def index_to_word(index: int):
    """
    returns the word of a given index
    Args:
        index: index of the word

    Returns:
        word: word of the given index. '.' if the index is 0 (end of sentence or </s>)
    """
    if index != 0:
      return words[index] 
    return '.'


def word_to_index(word: str):
    """
    returns the index of a given word
    Args:
        word: word of the given index. word should be normalized.

    Returns:
        index: index of the word. -1 if the word is not in the vocabulary
    """
    if word in indexes:
      return indexes[word]
    return -1


def state_to_sentence(state: list[int]):
    """
    returns the sentence of a given state
    Args:
        state: list of indexes of words

    Returns:
        sentence: string of words. '.' when the state is 0 (end of sentence or </s>)
    """
    res = ""
    for i in state:
      if i == 0:
        res = res[:-1]
        res+="."
        return res

      res +=  index_to_word(i) + " "
    return res


def sentence_to_state(sentence: str):
    """
    returns the state of a given sentence
    Args:
        sentence: string of words. sentence should be normalized.

    Returns:
        state: list of indexes of words. no need to add the index of </s> (end of sentence) to the state
    """
    states = list()
    sentence = normalize(sentence)
    for i in sentence.split():
      states.append(word_to_index(i))
    return states


print(index_to_word(10))
print(word_to_index('یک'))
print(state_to_sentence([390, 2884, 24, 0]))
print(sentence_to_state('من خوشحال هستم'))
print(state_to_sentence([390, 10, 791, 3816]))
print(sentence_to_state('من یک کتاب خریدم'))


یک
10
مقام کارت رئیس.
[389, 2887, 23]
مقام یک برقرار مهارت 
[389, 10, 787, 3808]


In [22]:
# example Q Table
q_table = {
    word_to_index('من'): (10, {
        word_to_index('خوشحال'): (20, {
            word_to_index('هستم'): (25, {
                0: (0, {}),
            }),
        }),
        word_to_index('یک'): (5, {
            word_to_index('کتاب'): (15, {
                word_to_index('خریدم'): (10, {}),
            }),
            word_to_index('گل'): (15, {
                word_to_index('دیدم'): (8, {}),
            }),
        })
    }),
    word_to_index('تو'): (10, {
        word_to_index('خوشحال'): (20, {
            word_to_index('هستی'): (7, {
                0: (0, {}),
            }),
        }),
        word_to_index('دو'): (5, {
            word_to_index('کتاب'): (15, {
                word_to_index('خریدی'): (11, {}),
            }),
        })
    }),
}
print('Q[من]', q_table[word_to_index('من')][0])
print('Q[من, خوشحال]', q_table[word_to_index('من')]
      [1][word_to_index('خوشحال')][0])
print('Q[من, خوشحال, هستم]', q_table[word_to_index('من')][1]
      [word_to_index('خوشحال')][1][word_to_index('هستم')][0])
q_table


Q[من] 10
Q[من, خوشحال] 20
Q[من, خوشحال, هستم] 25


{389: (10,
  {2887: (20, {-1: (25, {0: (0, {})})}),
   10: (5, {787: (15, {-1: (10, {})}), 479: (15, {-1: (8, {})})})}),
 2172: (10,
  {2887: (20, {7601: (7, {0: (0, {})})}),
   27: (5, {787: (15, {-1: (11, {})})})})}

### Hyperparameters
You can change these parameters to get better results.

In [23]:
q_table = {}
alpha = 0.8
gamma = 0.95
state_N = 6
N = 75

### Q-Learning Utility Functions (50 Points)

In [24]:
import random
import bisect
import math

weights = [1 for i in range(10000)]
random_choose = [i for i in range(10000)]

def random_index():
    """
    returns a random index based on the weights

    Returns:
        index: index of the word
    """
    return random.choices(random_choose, weights)[0]

In [25]:
def q_table_max_find(q_table: dict[int, tuple[int, dict]], state: list[int]):
    """
    returns the index of the word with the maximum Q value in the given state. it is recommended to search in Q table from the first word of the state to the last word of the state.
    if a word is not found in the Q table, you should search in the Q table of the next word of the state and so on.
    so if we don't have Q[W_1W_2...W_N], we search for Q[W_2W_3...W_N] and so on until Q[W_N]. if we don't have Q[W_N], we should return a random index.

    Args:
        q_table: Q table
        state: list of indexes of words

    Returns:
        index: index of the word with the maximum Q value in the given state. random index if the state is not in the Q table.
    """
    max_value = -math.inf
    max_index = 0
    current_q_table = q_table
    for i in range(len(state)):

      not_find = False
      for j in range(i,len(state)):
        if state[j] in current_q_table:
          current_q_table = current_q_table[state[j]][1]
        else:
          not_find = True
          break
      
      if not_find :
        current_q_table = q_table
        continue

      else:
        for q in current_q_table:
          if max_value <= current_q_table[q][0]:
            max_value = current_q_table[q][0]
            max_index = q
        break

    if max_index == 0 :
      max_index = random_index()
    
    return max_index






def q_table_update(q_table: dict[int, tuple[int, dict]], state: list[int]):
    """
    updates the Q table based on the given state. update the Q[W_1W_2...W_N] using the following formula:
    Q(s,a) += alpha * (reward + gamma * max_a' Q(s',a') - Q(s,a))
    where s is the state, a is the action, a' is the next action, s' is the next state, reward is the reward of the state, alpha is the learning rate, gamma is the discount factor.
    then update the Q[W_1W_2...W_{N-1}] and so on until Q[W_1].
    
    Args:
        q_table: Q table
        state: list of indexes of words
    """

    for n in range(len(state)-1,-1,-1):
      current_q_table = q_table
      for i in range(n):
        if state[i] not in current_q_table:
          current_q_table[state[i]] = (0,{})
        current_q_table = current_q_table[state[i]][1]     
      if state[n] not in current_q_table:
        current_q_table[state[n]] = (0, {})
      ## alan q table ta tah raft jelo
      #hala berim adad bedim
      maximum = -math.inf
      for i in current_q_table:
        maximum = max(maximum,current_q_table[i][0])
      #max peida shod
      rew = alpha * (reward(state_to_sentence(state[0:n]), state_to_sentence([state[n]])) + gamma * maximum -current_q_table[state[n]][0]) + current_q_table[state[n]][0]
      current_q_table[state[n]] = (rew , current_q_table[state[n]][1])












### Training Loop (10 Points)
Since search space is really big, we can let our model train for an hour or two and get a good result.

In [26]:
episodes = 10000
epsilon = 1
episode_N = 75
for ep in tqdm(range(episodes)):
  state = []
  for i in range(episode_N):
    if random.random() < epsilon:
       state.append(random_index())
    else:
      state.append(q_table_max_find(q_table, state))
    # to avoid infinite loop
    if len(state) > 1 and state[-1] == state[-2]:
      break
    q_table_update(q_table, state)
    if len(state) > state_N:
      state = state[1:]
    if state[-1] == 0:
      break
  epsilon *= 0.99975

100%|██████████| 10000/10000 [1:36:15<00:00,  1.73it/s]


### Testing (10 Points)
This will be the final output of our model. score will be based on how well the output fits with the corpus. Generated sentences should have some meaning in the neighborhood of each word.

In [27]:
def get_result(state, steps=75):
    for i in range(steps):
        state.append(q_table_max_find(q_table, state))
        if state[-1] == 0:
            break
        if len(state) > state_N:
            state = state[1:]
        yield state[-1]

state = sentence_to_state('ما')
print('ما', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()
state = sentence_to_state('یک')
print('یک', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()
state = sentence_to_state('ایران')
print('ایران', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()

ما یک بازگشت عصبانی پهلو درغزه همراه نیویورک چوبک بازیگر نمود#نما تیم Africa بست#بند مانوئل افتخار ۲۵۰۰ جنگ Iran راهپیمائی سردبیر من سایپا جابجا اوگاندا مسئله مبین southern Saturday hospital گذراند#گذران دگرگون پایگاه مالکیت elections پیروزی ابرازداشت افسر مذکور زغال نخستین گذاردند کرد#کن کند Monday مصالحه غرب طول شنود نخستین درآمدند داد میدهند باقیست فرستاد#فرست ایران مشکل بنادر برگزارکردند ترا یر مى‌شود پرندگان market برخوردارند سارس جمعیت دراوکراین شاید گزیده گشت تلسکوپ شرائط ۳۳ rebels افغان 
یک روزنامه دوشنبه ایستاد#ایست نبیل دیترویت میباشد درانتخابات هشدارداد Turkey برندگان بهترین چپگرا ۸۹ میراند ساحل مبنی Day روی صفحه قاطع اسکار از پوشیدن شدت ام عمده منقضی مسوول ببندد اخیرا آغازکند نگاهدارد Thursday برخاست ظاهرا تعلیم جزایر منجرشود تظاهرات ورزشکار راک پادشاه اندونزی کشت#کش مضمون درآمده‌اند مرگبار خرید#خر مؤثر تایوان قهرمانانه ازدست ببیند باخت#باز دشمن قبرس بازگشت بازگرداند احداث دید#بین پوشید#پوش خانه دار دما احمد گردیده‌اند Iraq داد#ده گذاردند برآید ببخشد سازنده میکوشد گشت#گرد ن